In [1]:
# from selenium import webdriver
import subprocess
from bs4 import BeautifulSoup
from collections import defaultdict
import json
from pprint import pprint

def cleanSent(sent):
    sent = sent.replace('\n','').replace('  ','')
    sent = sent.strip()
    return sent

def showTime():
    import datetime
    now = datetime.datetime.now()
    print('Now: %d:%d:%d\n' % (now.hour, now.minute, now.second))
    
CamDict = defaultdict(lambda: defaultdict(lambda:list()))
# option = webdriver.ChromeOptions()
# option.add_argument('headless')
# driver = webdriver.Chrome(executable_path=r'/usr/local/bin/chrome_driver', chrome_options=option)

V = "A word that describes an action, condition or experience."
N = "A word that refers to a person, place, idea, event or thing."
ADJ = "A word that describes a noun or pronoun."
targetPos = {'V', 'N', 'ADJ'}

In [8]:
def getEx(word):
    def get_runon():
        url = 'https://dictionary.cambridge.org/zht/搜索/english-chinese-traditional/direct/?q='+word
        try:
            status, msg = subprocess.getstatusoutput("curl -L "+url)
            soup = BeautifulSoup(msg, 'html.parser')
        except:
            return (word, 'fail')
            try:
                driver.get(url)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
            except:
                return word
        ans = word
        for child in soup.find_all("div", class_="runon"):
            try:
                pos = child.find('span', class_="pos")['title'].replace(V, 'V').replace(N, 'N').replace(ADJ, 'ADJ')
            except:
                continue
            if child.find("h3", class_="runon-title").text != word or pos not in targetPos:
                continue
            examps = child.find_all('div', class_="examp emphasized")
            for examp in examps:
                chTran = ''
                if language == 'ch':
                    try:
                        chTran = cleanSent(examp.find('span', class_="trans").text)
                    except:
                        chTran = ''
                try:
                    enExamp = cleanSent(examp.find('span', class_="eg").text)
                except:
                    continue
                exampDict[word][pos].append((enExamp, chTran))
                ans = json.loads(json.dumps(exampDict))
        return ans
    def enCam():
        url = 'https://dictionary.cambridge.org/dictionary/english/'+word
        try:
            status, msg = subprocess.getstatusoutput("curl "+url)
            soup = BeautifulSoup(msg, 'html.parser')
        except:
            return (word, 'fail'), ''
            try:
                driver.get(url)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
            except:
                return (word, 'fail'), ''
        if not soup.find('span', class_="hw") or soup.find('span', class_="hw").text != word:
            return word, ''
        return 'success!', soup
    
    def check_pos():
        pos_divs = soup.find_all('div', class_="entry-body__el")
        for pos_div in pos_divs:
            pos = pos_div.find('span', class_="pos")
            if not pos: continue
            try:
                pos = pos['title'].replace(V, 'V').replace(N, 'N').replace(ADJ, 'ADJ')
            except:
                continue
            if exampDict[word][pos]: continue # ???
            if pos not in targetPos: continue

            for child in pos_div.find_all("div", class_="runon"):
                if child.find("h3", class_="runon-title").text != word or child.find('span', class_="pos")['title'].replace(V, 'V').replace(N, 'N').replace(ADJ, 'ADJ') not in targetPos:
                    child.decompose()
            examps = pos_div.find_all('div', class_="examp emphasized")
            for examp in examps:
                chTran = ''
                if language == 'ch':
                    try:
                        chTran = cleanSent(examp.find('span', class_="trans").text)
                    except:
                        continue
                try:
                    enExamp = cleanSent(examp.find('span', class_="eg").text)
                except:
                    continue
                exampDict[word][pos].append((enExamp, chTran))
        
    language = 'ch'
    exampDict = defaultdict(lambda: defaultdict(lambda:list()))
#     url = 'https://dictionary.cambridge.org/zht/詞典/英語-漢語-繁體/'+word
    url = 'https://dictionary.cambridge.org/zht/搜索/english-chinese-traditional/direct/?q='+word
    try:
        status, msg = subprocess.getstatusoutput("curl -L "+url)
        soup = BeautifulSoup(msg, 'html.parser')
    except:
        return (word, 'fail')
        try:
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
        except:
            language = 'en'
            en_out, soup = enCam()
            if en_out != 'success!': return en_out
            
    # 跳頁
    if not soup.find('span', class_="hw") or soup.find('span', class_="hw").text != word:
        if language == 'en': return get_runon()
        language = 'en'
        en_out, soup = enCam()
        if en_out != 'success!': return get_runon()
    
    # 確認詞性
    check_pos()
    
    if 'N' in exampDict[word].keys() and 'V' in exampDict[word].keys() and 'ADJ' in exampDict[word].keys():
        return json.loads(json.dumps(exampDict))
    elif language == 'ch':
        language = 'en'
        en_out, soup = enCam()
        if en_out != 'success!': return en_out
        # 確認詞性
        check_pos()
    
    return json.loads(json.dumps(exampDict))


In [3]:
key_set = {'underperform', 'underperformance'}

In [5]:
# from joblib import Parallel, delayed

showTime()
pprint(getEx('aap'))
showTime()

Now: 14:47:24

'aap'
Now: 14:47:26



In [10]:
from joblib import Parallel, delayed

words = eval(open("words_need_delete.txt", "r").read())

existWord = set()
# showTime()
results = (Parallel(n_jobs=20, verbose=2)(delayed(getEx)(word) for word in words))

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   13.1s
[Parallel(n_jobs=20)]: Done 325 tasks      | elapsed:   28.3s
[Parallel(n_jobs=20)]: Done 608 tasks      | elapsed:   52.3s
[Parallel(n_jobs=20)]: Done 973 tasks      | elapsed:  1.5min
[Parallel(n_jobs=20)]: Done 1418 tasks      | elapsed:  2.3min
[Parallel(n_jobs=20)]: Done 1945 tasks      | elapsed:  3.3min
[Parallel(n_jobs=20)]: Done 2552 tasks      | elapsed:  4.4min
[Parallel(n_jobs=20)]: Done 3241 tasks      | elapsed:  5.8min
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:  7.4min
[Parallel(n_jobs=20)]: Done 4861 tasks      | elapsed:  9.1min
[Parallel(n_jobs=20)]: Done 5792 tasks      | elapsed: 11.2min
[Parallel(n_jobs=20)]: Done 6805 tasks      | elapsed: 13.1min
[Parallel(n_jobs=20)]: Done 7898 tasks      | elapsed: 15.3min
[Parallel(n_jobs=20)]: Done 9073 tasks      | elapsed: 17.5min
[Parallel(n_jobs=20)]: Done 10328 tasks      | elapsed: 19.9

In [11]:
for r in results:
    if type(r) == dict: CamDict.update(r)
    
    
file = open('camDict.txt', 'w')
data = json.loads(json.dumps(CamDict))
file.write(str(data))
file.close()

In [7]:
len(CamDict.keys())

23352

In [15]:
CamDict['store']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>>, {})

In [16]:
file = open('webdriver.txt', 'w')
file.write(str([ w for w in words if w not in CamDict.keys() ]))
file.close()


In [1]:
file = open('camDict.txt', 'r')
CamDict = eval(file.read())
file.close()

In [2]:
file = open('add.txt', 'r')
add = eval(file.read())
file.close()

In [6]:
CamDict.update(add)

In [9]:
file = open('camDict.txt', 'w')
#data = json.loads(json.dumps(CamDict))
file.write(str(CamDict))
file.close()